In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import h3
import numpy as np
import contextily as ctx
import matplotlib.pyplot as plt

from glob import glob
import os

from shapely.geometry import Polygon, MultiPolygon, Point
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [13]:

def load_data(idx=0):

    # List folders in the data directory
    folders = glob('data/Proyecto*/*')
    folders.sort()
    print(folders)

    # show them
    folders
    
    gdf = gpd.read_file(folders[idx])
    gdf.to_crs(epsg=4326, inplace=True)
    return gdf

def get_hexagons(gdf, APERTURE_SIZE=8, hex_col='hex_id'):
    total_bounds = gdf.bounds
    minx, miny, maxx, maxy = total_bounds.minx.min(), total_bounds.miny.min(), total_bounds.maxx.max(), total_bounds.maxy.max()
    bounding_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])

    # Get the hexagons
    h3_filled_polygon = pd.DataFrame(
        data = list(
            h3.polygon_to_cells(
                h3.LatLngPoly(bounding_polygon.exterior.coords),
                res=APERTURE_SIZE,
            )),
        columns=[hex_col]
    )
    h3_filled_polygon['geometry'] = h3_filled_polygon[hex_col].apply(lambda x: Polygon(h3.cell_to_boundary(x)))
    hex_list = gpd.GeoDataFrame(h3_filled_polygon, geometry='geometry')
    hex_list.set_crs(gdf.crs, inplace=True)
    return hex_list

def set_landuses_data(gdf):
    land_uses = gdf.copy()
    land_uses.to_crs('EPSG:32718', inplace=True)
    land_uses['id'] = land_uses.index
    lu_cols = ['id', 'DESTINO', 'geometry']
    land_uses = land_uses[lu_cols]
    land_uses['area_predio'] = land_uses.area
    land_uses.to_crs('EPSG:4326', inplace=True)
    return land_uses

def calculate_diversity(land_uses, hex_list, hex_col):
    gdf_overlay = gpd.overlay(hex_list, land_uses, how='intersection', keep_geom_type=False)
    gdf_overlay.to_crs('EPSG:32718', inplace=True)
    gdf_overlay['area_interseccion'] = gdf_overlay.area
    gdf_overlay.to_crs('EPSG:4326', inplace=True)    

    gdf_group_by_use = gdf_overlay.groupby([hex_col, 'DESTINO']).agg({'area_interseccion':'sum'}).reset_index().rename(columns={'area_interseccion': 'area_by_use'})
    gdf_group_by_hex = gdf_overlay.groupby([hex_col]).agg({'area_interseccion':'sum'}).reset_index().rename(columns={'area_interseccion': 'area_used_by_hex'})
    
    gdf_group = pd.merge(gdf_group_by_hex, gdf_group_by_use, on=hex_col, how='left')
    gdf_group['fraction_by_use'] = gdf_group['area_by_use'] / gdf_group['area_used_by_hex']
    gdf_group['info_by_use'] = -1*gdf_group['fraction_by_use']*np.log2(gdf_group['fraction_by_use'])
    gdf_group.loc[gdf_group['fraction_by_use']==1, 'info_by_use'] = 0
    gdf_group = gdf_group[[hex_col, 'info_by_use']].groupby(hex_col).agg({'info_by_use':'sum'}).reset_index().rename(columns={'info_by_use': 'diversity'})

    gdf_diversity = pd.merge(gdf_group, hex_list, on=hex_col)
    # gdf_diversity.drop(columns=['area_hex'], inplace=True)
    gdf_diversity = gpd.GeoDataFrame(gdf_diversity, geometry='geometry')
    return gdf_diversity

def plot_diversity(gdf_diversity, vmin, vmax, title, central_point, radius_vision=None, cmap='jet'):
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    gdf_diversity.plot(column='diversity', ax=ax, legend=True, vmin=vmin, vmax=vmax, cmap=cmap)
    # Convert central_point to the same CRS as gdf_diversity
    if isinstance(central_point, tuple):
        central_point = gpd.GeoSeries([Point(central_point)], crs='EPSG:4326')
    central_point = central_point.to_crs(gdf_diversity.crs)
    central_point.to_crs('EPSG:32718', inplace=True)

    if radius_vision is None:
        radius_vision = 100
    # Buffer the central point by the radius of vision
    vision_area = central_point.buffer(radius_vision)
    vision_area = gpd.GeoDataFrame(geometry=[vision_area])
    vision_area.set_crs('EPSG:32718', inplace=True)
    vision_area = vision_area.to_crs('EPSG:4326', inplace=True)
    # Plot the vision area
    vision_area.boundary.plot(ax=ax, color='red')

    # Set the limits of the plot to the vision area
    minx, miny, maxx, maxy = vision_area.total_bounds
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik ,crs=gdf_diversity.crs.to_string())

    ax.set_title(title)
    # fig.savefig(f'{title}.png')
    plt.plot()

In [14]:
# Define the aperture size
APERTURE_SIZE = 10
hex_col = f'h3_{APERTURE_SIZE}'

states = {
    'Actual': 0,
    'Futuro': 1,
}

central_point = (-36.710799929932236, -73.11414339114647)
divs = []
for state, idx in states.items():
    gdf = load_data(idx)
    hex_list = get_hexagons(gdf, APERTURE_SIZE=APERTURE_SIZE, hex_col=hex_col)
    land_uses = set_landuses_data(gdf)
    gdf_diversity = calculate_diversity(land_uses, hex_list, hex_col)
    divs.append(gdf_diversity)

['data/Proyecto La Poza/Actual', 'data/Proyecto La Poza/Futuro']
['data/Proyecto La Poza/Actual', 'data/Proyecto La Poza/Futuro']


In [ ]:
for div, state in zip(divs,states.keys()):
    div.to_file(f'data/{state}_diversity')

In [ ]:
import shutil

# Path to the folder containing the results
results_folder = 'data/*_diversity'

# Name of the zip file to create
zip_file_name = 'results.zip'

# Compress the folder into a zip file
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', results_folder)

'/home/diego/Proyectos/clbb-modules/indicators/land_uses_diversity_map/results.zip'

In [15]:
import matplotlib.colors as mp_color
import matplotlib.cm as cmx
import pydeck as pdk

In [20]:
def create_custom_cmap(values):
    # Define the colors for the colormap
    colors = ['#42f5da', '#7DDA58', '#DECE58', '#DE8758', '#DE5858']

    # Create the colormap object
    cmap = mp_color.ListedColormap(colors)

    # Define the color boundaries for the colormap
    boundaries = values

    # Create the normalization object
    norm = mp_color.BoundaryNorm(boundaries, len(colors))

    return cmap, norm

def get_color_value(value, cmap, norm):
    # Normalize the value to the range of the colormap
    norm_value = norm(value)

    # Get the color value from the colormap
    color = cmap(norm_value)

    scaled_colors = list(map(lambda x: int(x*255), color[:3]))
    scaled_alpha = int(color[3]*100)

    scaled_colors += [scaled_alpha]
    return scaled_colors


def colormap_dataframe(df, value_col, cmap, norm, max_val=None):

    xdf = df.copy()
    xdf['color'] = xdf[f'{value_col}'].apply(lambda x: get_color_value(x, cmap, norm))
    xdf[['R', 'G', 'B', 'A']] = pd.DataFrame(xdf['color'].to_list())
    xdf = xdf.drop(['color'], axis=1)

    xdf[value_col] = xdf[value_col].round(1)

    return xdf

def create_h3_hex_layer(df, hex_col):

    # Define a layer to display on a map
    layer = pdk.Layer(
        "H3HexagonLayer",
        df,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=True,
        get_hexagon=hex_col,
        get_fill_color="[R, G, B, A]"
    )

    return layer

In [22]:
div = gpd.read_file('data/Actual_diversity')

print(div)

bbox = div.total_bounds
minx, miny, maxx, maxy = bbox
bounding_box_points = [
    (minx, miny),  # Esquina inferior izquierda
    (maxx, miny),  # Esquina inferior derecha
    (maxx, maxy),  # Esquina superior derecha
    (minx, maxy)   # Esquina superior izquierda
]
bounding_box_points

# Crear un polígono a partir de los puntos
bbox_polygon = Polygon(bounding_box_points)

# Crear un GeoDataFrame a partir del polígono
bbox = gpd.GeoDataFrame(index=[0], geometry=[bbox_polygon], crs="EPSG:4326")

## Apply the colormap to the dataframe
cmap, norm = create_custom_cmap(np.linspace(0, 3, 5))
print(cmap, norm)
xdf = colormap_dataframe(df=div, value_col='diversity', cmap=cmap, norm=norm)
print(xdf)

## Now create a H3HexagonLayer
layer = create_h3_hex_layer(xdf, hex_col)
print(layer)

# Set the viewport location
x,y = bbox.centroid.x.values[0], bbox.centroid.y.values[0]
view_state = pdk.ViewState(latitude=y, longitude=x, zoom=11, bearing=0, pitch=45)

tooltip = {
        # "html": "{display_text}",
        # "style": {
        #     "backgroundColor": "#4CAF50",   # Green shade for background
        #     "color": "#FFFFFF",             # White for text color
        #     "border": "2px solid #4CAF50",  # Matching border color
        #     "borderRadius": "5px",          # Rounded corners
        #     "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
        # }
    }

# # Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip)
r.to_html('h3_hex_layer.html', iframe_width=900)

                h3_10  diversity  \
0     8aee0e036007fff   0.000000   
1     8aee0e03600ffff   0.661072   
2     8aee0e036017fff   0.000000   
3     8aee0e03601ffff   0.000000   
4     8aee0e036027fff   0.338425   
...               ...        ...   
1294  8aee0e1aeb8ffff   0.000000   
1295  8aee0e1aeb97fff   0.653864   
1296  8aee0e1aeb9ffff   0.000000   
1297  8aee0e1aeba7fff   0.000000   
1298  8aee0e1aebb7fff   1.324335   

                                               geometry  
0     POLYGON ((-73.05633 -36.77461, -73.05597 -36.7...  
1     POLYGON ((-73.05518 -36.77671, -73.05483 -36.7...  
2     POLYGON ((-73.05637 -36.77015, -73.05602 -36.7...  
3     POLYGON ((-73.05522 -36.77225, -73.05487 -36.7...  
4     POLYGON ((-73.05743 -36.77698, -73.05708 -36.7...  
...                                                 ...  
1294  POLYGON ((-73.11346 -36.69636, -73.11311 -36.6...  
1295  POLYGON ((-73.11465 -36.68978, -73.1143 -36.68...  
1296  POLYGON ((-73.1135 -36.69189, -73.11315